In [1]:
#phase correlation libraries
import numpy as np
from scipy import misc
from argparse import ArgumentParser

#complementary labraries
# from google.colab import drive
import sys
import os
import matplotlib.pyplot as plt
import cv2
import pandas as pd

In [2]:
# drive.mount('/content/drive', force_remount=True)

sys.path.append('../codes/')
from utils import video_utils

Mounted at /content/drive


In [3]:
def scale_img(src, scale):
    # calculate new dimensions
    new_w = int(src.shape[1] * scale)
    new_h = int(src.shape[0] * scale)

    return cv2.resize(src, (new_w, new_h))

def phase_correlation(img, img_offset, scale=None):
    """Image translation registration by cross-correlation.
    It obtains an estimate of the cross-correlation peak by an FFT.
    It is very similar to `skimage.feature.register_translation`.
    However, our function runs faster because we restrict it to our application.
    Args:
        img (array): image.
        img_offset (array): offset image. Must have the same dim as `img`.
        scale (float, optional): If not `None`, rescale input images to run faster.
        Defaults to None.
    Returns:
        array:  shift vector (in pixels) required to register `img_offset` with
        `img`.  Axis ordering is consistent with numpy (e.g. Z, Y, X)
    """

    if img.shape != img_offset.shape:
        raise ValueError("Error: images must be same size")

    img = np.float32(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
    img_offset = np.float32(cv2.cvtColor(img_offset, cv2.COLOR_BGR2GRAY))

    if scale is not None:
        img = scale_img(img, scale)
        img_offset = scale_img(img_offset, scale)

    (x, y), c = cv2.phaseCorrelate(img, img_offset)

    if scale is not None:
        x /= scale
        y /= scale

    return (x, y), c

## ARRUMAR OS NOMES DOS FRAMES DE TUBIACANGA ANTES DE UTILIZAR A REGIÃO


In [4]:
TUBIACANGA_PATH = '/content/drive/My Drive/repo/tubiacanga/'
PACIENCIA_PATH  = '/content/drive/My Drive/repo/paciencia/'
SEPETIBA_PATH   = '/content/drive/My Drive/repo/sepetiba/'
SANTA_CRUZ_PATH = '/content/drive/My Drive/repo/santa_cruz/'

ALL             = 'frames/all/'
OVERLAPED       = 'frames/overlaped/'

In [14]:
def phase_correlate_dataFrame(frames_path):
  all_path = frames_path + ALL
  overlaped_path = frames_path + OVERLAPED
  frames_count = 0
  for path in os.listdir(frames_path):
    if os.path.isfile(os.path.join(frames_path, path)):
      frames_count += 1

  r_array = []
  x = []
  y = []
  c = []
  counter = 0
  frameB = cv2.imread(all_path + 'frame_{}.jpeg'.format('{:05d}').format(counter))
  while counter < frames_count:
    counter += 1
    frameA = frameB
    frameB = cv2.imread(all_path + 'frame_{}.jpeg'.format('{:05d}').format(counter))
    results = phase_correlation(frameA, frameB, scale = 0.4)
    r_array.append(results)
    x.append(results[0][0])
    y.append(results[0][1])
    c.append(results[1])

  x_dataFrame = pd.DataFrame(x)
  x_dataFrame.columns = ['x']
  y_dataFrame = pd.DataFrame(y)
  y_dataFrame.columns = ['y']
  c_dataFrame = pd.DataFrame(c)
  c_dataFrame.columns = ['c']
  r_dataFrame = pd.concat([x_dataFrame, y_dataFrame, c_dataFrame], axis=1)
  return r_dataFrame


12582
